In [ ]:
import os
import numpy as np
import pandas as pd
import json

from radiomics import featureextractor

import logging
# set level for all classes
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)


"""
높은 binWidth:
장점: 데이터 단순화, 계산 속도 향상, 일관성 증가
단점: 세밀한 텍스처 정보 손실 가능성
낮은 binWidth:
장점: 세밀한 텍스처 정보 보존
단점: 잡음 증가, 계산 부담 증가
"""

# Pyradiomics feature extractor 설정
params = {
    'binWidth': 25,  # 히스토그램의 각 빈의 너비를 25로 설정   
    'resampledPixelSpacing': None,
    'correctMask': True,
    'force2D': True,  # 2D 데이터 처리
    'label': 255,
    'normalize': True
    
}

### 특정 feature들에 대해서만 특징 추출
extractor = featureextractor.RadiomicsFeatureExtractor(**params)
extractor.disableAllFeatures()  # 모든 특징 비활성화
extractor.enableFeatureClassByName('shape2D')  # GLCM 특징 활성화
extractor.enableFeatureClassByName('firstorder')  # Firstorder 특징 활성화
extractor.enableFeatureClassByName('glcm')  # GLCM 특징 활성화

# 이미지와 마스크 경로

path_df = '/home/user/workdir/notices/data'
path_data = os.path.join(path_df, 'breast')
path_abimg = os.path.join(path_data, 'image', 'abnormal')

abnameList = sorted([name.split('.')[0] for name in os.listdir(os.path.join(path_data, 'label', 'json', 'abnormal')) if name.endswith('.json')])

mnList = list()
bnList = list()

In [ ]:
# 마스크가 2D일 경우 3D로 확장하는 함수
def prepare_image_and_mask(image_path, mask_path):
    # 이미지와 마스크 로드
    image = sitk.ReadImage(image_path)
    mask = sitk.ReadImage(mask_path)

    # 마스크가 2D인 경우 3D로 확장
    if mask.GetDimension() == 2:
        mask_array = sitk.GetArrayFromImage(mask)
        
        # 3D로 확장 (이미지의 슬라이스 개수에 맞춰 복제)
        mask_3d_array = np.repeat(mask_array[np.newaxis, :, :], image.GetSize()[2], axis=0)
        
        # 3D 마스크 SimpleITK 이미지로 변환
        mask = sitk.GetImageFromArray(mask_3d_array)
        mask.CopyInformation(image)  # 이미지의 공간 정보를 마스크에 복사

    return image, mask


data = []
# 메타 데이터와 특징 추출 반복문
for n, abname in enumerate(abnameList):
    # 메타 데이터로부터 양악성 클래스 추출
    with open(os.path.join(path_data, 'label', 'json', 'abnormal', f'{abname}.json'), 'r') as info_json:
        patient_info = json.load(info_json)
        abtype = patient_info['patientInfo']['lesion_type']
        if abtype == 'malignant':
            mnList.append(abname)
        elif abtype == 'benign':
            bnList.append(abname)
        else:
            print(f'Unknown: {abname}')

    imagePath = os.path.join(path_data, 'image', 'abnormal', f'{abname}.dcm')
    maskPath = os.path.join(path_data, 'label', 'nii', f'{abname}.nii')


    image, mask = prepare_image_and_mask(imagePath, maskPath)

    # pyradiomics를 통해 특징 추출
    result = extractor.execute(image, mask)
    features = {key: value for key, value in result.items() if key.startswith('original')}
    features['label'] = 1 if abtype == 'malignant' else 0  
    data.append(features)
    
    print(f'Success: {n+1}/{len(abnameList)}')

# 결과 데이터프레임 생성
df = pd.DataFrame(data)


In [ ]:
print(df)
path_save = os.path.join(os.path.abspath('.'), 'data')
os.makedirs(path_save, exist_ok=True)
df.to_csv(os.path.join(path_save, 'Radiomic_features_all.csv'), index=False)

In [ ]:
featureList = df.columns.to_list()
print(featureList)
print(len(featureList))
print(len(set(featureList)))